In [1]:
import json
import pandas as pd
import hdbscan
import numpy as np
from TFTClusterer import TFTClusterer
from datetime import datetime, timedelta
from googledrivesave import trashfolder, outputtodrive
import pygsheets
from SQLGatherer import loaddb
from matplotlib import pyplot as plt
df = loaddb(hours=10)
gameversion = df['game_version'].max()
df=df.loc[df['game_version']==gameversion]

assert len(df) >= 100, "less than 100 matches in newest patch"
#Cluster Data

clusterclass=TFTClusterer(df)
#clusterclass.imputetraits()
clusterclass.reduce_dimension()
clusterclass.cluster(divisor=20, cluster_selection_epsilon=0)

clusterclass.eval_clustering()
clusterclass.plot.figure.savefig('fig.png')
pd.DataFrame(clusterclass.clusterdf.groupby('hdbnumber')['hdb'].value_counts()).to_csv('hdbnumber.csv')
print(sum(clusterclass.clusterdf['hdbnumber']==0)/len(clusterclass.clusterdf))

#output Files for Power BI
clusterclass.unitshdb.to_csv("unitshdb.csv",index=False)
clusterclass.itemshdb.to_csv("itemshdb.csv",index=False)
clusterclass.clusterdf[["comp_id","participants.placement","hdb","game_variation"]].to_csv("hdb.csv",index=False)

#Write newest date
f = open('newestdate.csv','w')
f.write(str(datetime.fromtimestamp(df['game_datetime'].max()/1e3))) #Give your csv text here.
f.close()



HDB Scan


<Figure size 640x480 with 2 Axes>

<Figure size 1200x800 with 1 Axes>

0.30776043396435293


In [27]:
clusterclass.clusterdf['hdb'].value_counts()

Dark Star Protector    4497
Sniper Astro           1609
Brawler Battlecast     1318
Blademaster Chrono     1073
Cybernetic Chrono      1013
Vanguard Mystic         698
No Comp                 201
Name: hdb, dtype: int64

In [28]:
units

NameError: name 'units' is not defined

In [29]:
EXPECTED_PRICE = {
    1 : {
        3 : 10.4,
        4 : 10.4,
        5 : 10.4,
        6 : 10.4,
        7 : 10.4,
        8 : 10.4,
        9 : 10.4,},
    2 : {
        3 : 14.5,
        4 : 14.5,
        5 : 14.5,
        6 : 14.5,
        7 : 14.5,
        8 : 14.5,
        9 : 14.5,},
    3 : {
        3 : 96.8,
        4 : 32.8,
        5 : 20.0,
        6 : 16.8,
        7 : 15.4,
        8 : 14.5,
        9 : 14.5,},
    4 : {
        3 : 500,
        4 : 300,
        5 : 121, 
        6 : 36.8,
        7 : 24.8,
        8 : 18.8,
        9 : 12.8,},
    5 : {
        3 : 999,
        4 : 999,
        5 : 999,
        6 : 481,
        7 : 121,
        8 : 48.8,
        9 : 24.8,}
}

In [30]:
units['gold'] = units['rarity'] + 1
units.head(5)

NameError: name 'units' is not defined

In [31]:
units['ExpectedGold']=units.apply(
            lambda row:
            (3**((row['tier'] - 1))*1.15) * (row['gold'] + EXPECTED_PRICE[row['gold']][row['participants.level']]),
            axis = 1)

NameError: name 'units' is not defined

In [32]:
units['ExpectedGold'].describe()

NameError: name 'units' is not defined